# Final Project 
##  - Asset Management & Portfolio Construction

Group Members: 
Yuchen Wang & Chengyi Xu


## Table of Contents
- Overview
- Essential Libraries
- Data
- Exploratory Data Analysis
    - Remove unrelated companies
    - Find and remove irregular entries
    - Seperate the dataframe by ticker
    - Combine the same company with different ticker
    - Readjust the datapool
- Calculation of important data
- Portfolio Creation
- Plot the efficient frontier
- Portfolio Analysis
- Next Steps: Portfolio prediction testing

## Overview
Crude oil has been viewed as the single most important commodity in the world for its unreplaceable role in modern manufacturing industries. As a nonrenewable natural resource, crude oil is scarce in both economic and everyday definitions, so its price is expected to fluctuate with supply and demand on the market. Investors for businesses in the crude oil production industries see this fluctuation as both opportunities and potential risks. There is great interest in studying how to build a portfolio with smaller volatility but higher return. 
We take 10 publicly traded crude oil producers as our stock pool in our study. They are: 
- Occidental Petroleum (OXY)
- Continental Resources (CLR)
- Marathon Oil (MRO)
- EOG Resources (EOG)
- ConocoPhilips (COP)
- Diamondback Energy (FANG)
- Exxon Mobil Corporation (XOM)
- Chevron Corporation (CVX)
- Ovintiv Inc. (OVV)
- Pioneer Natural Resources Corporation (PXD)

We take the S&P500, Dow Jones Industrial Average, and Crude oil Nov 22 as the benchmarks to represent the market turbulence. 
To test the reliability of our rebalancing strategy, we will test out with out-of-sample data by using the “future” data from the set date of our study. 

## Essential Libraries

In [ ]:
import wrds
import pandas as pd
import numpy as np
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
import datetime
from datetime import datetime
import matplotlib.pyplot as plt
from dateutil.parser import parse

## Data
The crude oil industry is a mature market, and the companies in the industry have a long enough history for us to test our models. The industry itself experiences a lot of turbulence, and the thriving times and the depressed times appeared interchangeably, the pattern shown in the industry making it suitable for time series analysis. Shale oil, benefited from technology advancement, has a great potential in future production growth. The companies chosen are the top 10 shale oil producers in 2020, and three indices are used as benchmark predictors.

The data set is acquired mainly from three sources.
1. Historical trading data of the companies from CRSP dataset by WRDS
2. Historical financial reporting data of companies from Yahoo Finance Plus
3. Historical values of Index data from CRSP dataset by WRDS

We use historical daily data for seven top companies in the crude oil industry from 2002 to 2021.


In [ ]:
# acquiring risk-free rates from yahoo finance
# here, 10-year T-bond is used for risk free rate

import yfinance as yf
t_bond_df = yf.download('^TNX', start = '1990-01-02', end = '2022-03-31')

In [ ]:
t_bond_df.info()

In [ ]:
t_bond_df = t_bond_df.reset_index()
t_bond_df['TICKER'] = 'RF'
t_bond_df['PRC'] = t_bond_df['Close']
t_bond_df['COMNAM'] = "Risk_free"
t_bond_df['date'] = pd.to_datetime(t_bond_df['Date']).dt.strftime('%Y%m%d')
t_bond_df = t_bond_df[["TICKER", "PRC","date","COMNAM"]]
t_bond_df = t_bond_df.set_index("COMNAM")
t_bond_df['date'] = t_bond_df['date'].astype(int)

In [ ]:
t_bond_df.head(5)

In [ ]:
#import dataset
oil_df = pd.read_csv('/Users/chengyixu/Desktop/JHU/fall 2 computional/final 2/oil_comp.csv', low_memory = False)
#oil_df = pd.read_csv('F:\\Data_Learning\\computational_fin\\raw_data.csv', low_memory = False)

In [ ]:
#Basic summary
oil_df.head()

In [ ]:
# acquiring market return rates from yahoo finance
# here, S&P 500 is used for risk free rate
SP500_df = yf.download('^GSPC', start = '1990-01-02', end = '2022-03-31')

## Exploratory Data Analysis
Some of the companies changed their names and tickers due to operational changes and company mergers. The raw data pool combined them altogther. 

There are noise in the data needed to be removed. 

### Remove unrelated companies

In [ ]:
# Set the index of the DataFrame to the stock name
oil_with_index = oil_df.set_index("COMNAM")

In [ ]:
other_comp = [
    "COPLEY PROPERTY INC",
    "CLARION COMMERCIAL HOLDINGS INC",
    "COLOR SYSTEM TECHNOLOGY INC",
    "C S T ENTERTAINMENT IMAGING INC",
    "C S T ENTERTAINMENT INC",
    "CONSOLIDATED PRODUCTS INC",
    "STEAK N SHAKE CO",
    "BIGLARI HOLDINGS INC"
]

oil_with_index = oil_with_index.drop(other_comp)

### Remove unrelated columns

In [ ]:
# Clearing the unuseful columns
vis_col = ['date', 'TICKER', 'PRC', 'RET', 'VOL', 'BIDLO', 'ASKHI', 'RETX']

In [ ]:
oil_with_index = oil_with_index[vis_col]

### Find and remove irregular entries

In [ ]:
# Missing Values
def check_missing_values(df, df_name):
  print(f'Number of missing Values by Feature in {df_name}\n',df.isnull().sum())
  columns_with_NaNs = []
  dates_with_NaNs = []
  for col in df.columns:
    if df[col].isnull().sum() > 0:
      null_series = df[col].isnull()
      columns_with_NaNs.append(col)
      dates_with_NaNs.append(null_series[null_series].index)
  print('Columns with NaNs:',columns_with_NaNs)
  print('Columns with NaNs:',dates_with_NaNs)

In [ ]:
check_missing_values(oil_with_index, 'oil_with_index')

In [ ]:
oil_with_index = oil_with_index[oil_with_index['RET'] != 'C']

### Seperate the dataframe by ticker

In [ ]:
#CHV dataframe
CHV_df = oil_with_index[oil_with_index['TICKER'] == 'CHV']
#CLR dataframe
CLR_df = oil_with_index[oil_with_index['TICKER'] == 'CLR']
#COP dataframe
COP_df = oil_with_index[oil_with_index['TICKER'] == 'COP']
#COPI dataframe
COPI_df = oil_with_index[oil_with_index['TICKER'] == 'COPI']
#CVX dataframe
CVX_df = oil_with_index[oil_with_index['TICKER'] == 'CVX']
#ECA dataframe
ECA_df = oil_with_index[oil_with_index['TICKER'] == 'ECA']
#EOG dataframe
EOG_df = oil_with_index[oil_with_index['TICKER'] == 'EOG']
#FANG dataframe
FANG_df = oil_with_index[oil_with_index['TICKER'] == 'FANG']
#MRO dataframe
MRO_df = oil_with_index[oil_with_index['TICKER'] == 'MRO']
#OVV dataframe
OVV_df = oil_with_index[oil_with_index['TICKER'] == 'OVV']
#OXY dataframe
OXY_df = oil_with_index[oil_with_index['TICKER'] == 'OXY']
#P dataframe
P_df = oil_with_index[oil_with_index['TICKER'] == 'P']
#PCX dataframe
PCX_df = oil_with_index[oil_with_index['TICKER'] == 'PCX']
#PDP dataframe
PDP_df = oil_with_index[oil_with_index['TICKER'] == 'PDP']
#X dataframe
X_df = oil_with_index[oil_with_index['TICKER'] == 'X']
#XOM dataframe
XOM_df = oil_with_index[oil_with_index['TICKER'] == 'XOM']
#XON dataframe
XON_df = oil_with_index[oil_with_index['TICKER'] == 'XON']
#XON dataframe
PXD_df = oil_with_index[oil_with_index['TICKER'] == 'PXD']

### Explore data

In [ ]:
oil_with_index.groupby('TICKER')['date'].agg(['min','max']).sort_values(by=['min'])

In [ ]:
oil_with_index.groupby('COMNAM')['date'].agg(['min','max']).sort_values(by=['min'])

### Combine the same company with different ticker
We have observed that some of the companies changed their company names in the past, and changed their tickers at the same time. It does not make sense to see them as different companies. With the aggregated data grouped by company names and their tickers, we can combine them together so that it creates a continuous record under the same name and ticker. 

After the combination, log return is calculated at the same time.

In [ ]:
#combine the same company with differnet ticker
XOM_united = pd.concat([XOM_df, XON_df], axis=0)
XOM_united['TICKER'] = XOM_united['TICKER'].replace(['XON'], 'XOM')
XOM_united.sort_values(by=['date'])
XOM_united['RET'] = XOM_united['RET'].astype(float)
XOM_united['logreturn'] = np.log(1+XOM_united['RET'])

In [ ]:
#combine the same company with differnet ticker
PXD_united = pd.concat([PXD_df, PDP_df], axis=0)
PXD_united['TICKER'] = PXD_united['TICKER'].replace(['PDP'], 'PXD')
PXD_united.sort_values(by=['date'])
PXD_united['RET'] = PXD_united['RET'].astype(float)
PXD_united['logreturn'] = np.log(1+PXD_united['RET'])

In [ ]:
#combine the same company with differnet ticker
CVX_united = pd.concat([CVX_df, CHV_df], axis=0)
CVX_united['TICKER'] = CVX_united['TICKER'].replace(['CHV'], 'CVX')
CVX_united.sort_values(by=['date'])
CVX_united['RET'] = CVX_united['RET'].astype(float)
CVX_united['logreturn'] = np.log(1+CVX_united['RET'])

In [ ]:
#combine the same company with differnet ticker
MRO_united = pd.concat([MRO_df, X_df], axis=0)
MRO_united['TICKER'] = MRO_united['TICKER'].replace(['X'], 'MRO')
MRO_united.sort_values(by=['date'])
MRO_united['RET'] = MRO_united['RET'].astype(float)
MRO_united['logreturn'] = np.log(1+MRO_united['RET'])

In [ ]:
#combine the same company with differnet tickers
OVV_united = pd.concat([OVV_df, PCX_df, ECA_df], axis=0)
OVV_united['TICKER'] = OVV_united['TICKER'].replace(['PCX'], 'OVV')
OVV_united['TICKER'] = OVV_united['TICKER'].replace(['ECA'], 'OVV')
OVV_united.sort_values(by=['date'])
OVV_united['RET'] = OVV_united['RET'].astype(float)
OVV_united['logreturn'] = np.log(1+OVV_united['RET'])

In [ ]:
#combine the same company with differnet tickers
COP_united = pd.concat([COP_df, P_df], axis=0)
COP_united['TICKER'] = COP_united['TICKER'].replace(['P'], 'COP')
COP_united.sort_values(by=['date'])
COP_united['RET'] = COP_united['RET'].astype(float)
COP_united['logreturn'] = np.log(1+COP_united['RET'])

In [ ]:
EOG_df['RET'] = EOG_df['RET'].astype(float)
EOG_df['logreturn'] = np.log(1+EOG_df['RET'])

In [ ]:
OXY_df['RET'] = OXY_df['RET'].astype(float)
OXY_df['logreturn'] = np.log(1+OXY_df['RET'])

In [ ]:
FANG_df['RET'] = FANG_df['RET'].astype(float)
FANG_df['logreturn'] = np.log(1+FANG_df['RET'])

In [ ]:
CLR_df['RET'] = CLR_df['RET'].astype(float)
CLR_df['logreturn'] = np.log(1+CLR_df['RET'])

### Readjust the datapool

In [ ]:
#Combine all data frames of each stock together.
Oil_df_final = pd.concat([
    COP_united, 
    PXD_united, 
    XOM_united, 
    CVX_united, 
    MRO_united, 
    EOG_df, 
    OXY_df, 
    FANG_df, 
    OVV_united, 
    CLR_df,
    t_bond_df
    ], axis=0)

Oil_df_final

In [ ]:
Oil_df_final.groupby('TICKER')['date'].agg(['min','max']).sort_values(by=['min'])

In [ ]:
#design a function to select how many years to test
def decided_period(n):
    max_date = 20220331
    decided_date = 20220331 - n*10000
    return decided_date

Oil_df_final_2 = Oil_df_final[(Oil_df_final['date'] >= decided_period(3))]

In [ ]:
#Adjust the data
SP500_df = SP500_df.reset_index()
SP500_df['TICKER'] = 'MKT'
SP500_df['PRC'] = SP500_df['Close']
SP500_df['COMNAM'] = "Market_Return"
SP500_df['date'] = pd.to_datetime(SP500_df['Date']).dt.strftime('%Y%m%d')
SP500_df = SP500_df[["TICKER", "PRC","date","COMNAM"]]
SP500_df = SP500_df.set_index("COMNAM")
SP500_df['date'] = SP500_df['date'].astype(int)
SP500_df_2 = SP500_df[(SP500_df['date'] > decided_period(3))]

In [ ]:
#Find the Market Return
MKT = SP500_df_2["PRC"].pct_change().apply(lambda x: np.log(1 + x)).sum()

In [ ]:
#keep it clearer
Oil_short = Oil_df_final_2[["TICKER", "PRC","date"]]
Oil_short

In [ ]:
#create pivot table
oil_table = Oil_short.pivot_table(index='date', columns='TICKER', values='PRC', aggfunc='sum').sort_values('date', ascending=True)
oil_table

## Calculation of important data

In [ ]:
#construct cov matrix, it is log metric here.
cov_matrix = oil_table.pct_change().apply(lambda x: np.log(1+x)).cov()
cov_matrix

In [ ]:
#corr matrix, log here
corr_matrix = oil_table.pct_change().apply(lambda x: np.log(1+x)).corr()
corr_matrix

In [ ]:
#Calculate the Betas
betas = cov_matrix['RF'] / cov_matrix.loc['RF','RF']
betas

In [ ]:
#calculate period returns
ind_er = oil_table.pct_change().apply(lambda x: np.log(1+x)).sum()
ind_er

In [ ]:
#find peiod std, need work
ann_sd = oil_table.pct_change().apply(lambda x: np.log(1+x)).std().apply(lambda x: x*np.sqrt(252*3))
ann_sd

In [ ]:
# create a table for this.
assets = pd.concat([ind_er, ann_sd], axis=1) # Creating a table for visualising returns and volatility of assets
assets.columns = ['Returns', 'Volatility']
assets

In [ ]:
oil_with_index.iloc[:,[0,1]]

In [ ]:
oil_with_index.mean()

In [ ]:
oil_with_index.groupby('TICKER')['date'].max()

# Portfolio Creation

In [ ]:
#create the portofilio

p_ret = [] # Define an empty array for portfolio returns
p_vol = [] # Define an empty array for portfolio volatility
p_weights = [] # Define an empty array for asset weights

num_assets = len(oil_table.columns)
num_portfolios = 1000

for portfolio in range(num_portfolios):
    weights = []
    for portfolio in range(num_portfolios):
        weight = np.random.uniform(-3, 3, num_assets)
        weight = weight/np.sum(weight)
        if all(value < 3 and value > -3 for value in weight):
            weights = weight
    p_weights.append(weights)
    returns = np.dot(weights, ind_er) # Returns are the product of individual expected returns of asset and its 
                                      # weights 
    p_ret.append(returns)
    var = cov_matrix.mul(weights, axis=0).mul(weights, axis=1).sum().sum()# Portfolio Variance
    sd = np.sqrt(var) # Daily standard deviation
    ann_sd = sd*np.sqrt(252) # Annual standard deviation = volatility
    p_vol.append(ann_sd)

data = {'Returns':p_ret, 'Volatility':p_vol}

for counter, symbol in enumerate(oil_table.columns.tolist()):
    #print(counter, symbol)
    data[symbol+' weight'] = [w[counter] for w in p_weights]

In [ ]:
#adjust the portofolio with more metrics
portfolios  = pd.DataFrame(data)
portfolios['Sharpe'] = (portfolios['Returns'] - ind_er.iloc[-2])/ portfolios['Volatility']

portfolios['Treynor'] = (portfolios['Returns'] - ind_er.iloc[-2])/ (
    portfolios['CLR weight'] * betas.iloc[0] +
    portfolios['COP weight'] * betas.iloc[1] +
    portfolios['CVX weight'] * betas.iloc[2] +
    portfolios['EOG weight'] * betas.iloc[3] +
    portfolios['FANG weight'] * betas.iloc[4] +
    portfolios['MRO weight'] * betas.iloc[5] +
    portfolios['OVV weight'] * betas.iloc[6] +
    portfolios['OXY weight'] * betas.iloc[7] +
    portfolios['PXD weight'] * betas.iloc[8] +
    portfolios['RF weight'] * betas.iloc[9] +
    portfolios['XOM weight'] * betas.iloc[-1])

portfolios['Jensen'] = portfolios['Returns'] - ind_er.iloc[-2] - (
    portfolios['CLR weight'] * betas.iloc[0] +
    portfolios['COP weight'] * betas.iloc[1] +
    portfolios['CVX weight'] * betas.iloc[2] +
    portfolios['EOG weight'] * betas.iloc[3] +
    portfolios['FANG weight'] * betas.iloc[4] +
    portfolios['MRO weight'] * betas.iloc[5] +
    portfolios['OVV weight'] * betas.iloc[6] +
    portfolios['OXY weight'] * betas.iloc[7] +
    portfolios['PXD weight'] * betas.iloc[8] +
    portfolios['RF weight'] * betas.iloc[9] +
    portfolios['XOM weight'] * betas.iloc[-1])* (MKT - ind_er.iloc[-2])

portfolios.sort_values(by=['Sharpe'], ascending = False)
# Dataframe of the 10000 portfolios created
# the first one provides the highest Sharpe Ratio

## Plot the efficient frontier

In [ ]:
#plot the efficient frontier
portfolios.plot.scatter(x='Volatility', y='Returns', marker='o', s=10, alpha=0.3, grid=True, figsize=[10,10])
plt.xlim(0, 10)
plt.ylim(-5, 20)
plt.show()

In [ ]:
#Create Rankings for these methodology
portfolios['Vol_rank'] = portfolios['Volatility'].rank(method='min')
portfolios['Sharpe_rank'] = portfolios['Sharpe'].rank(ascending=False)
portfolios['Trey_rank'] = portfolios['Treynor'].rank(ascending=False)
portfolios['Jensen_rank'] = portfolios['Jensen'].rank(ascending=False)
portfolios['Highest_rank'] = portfolios['Jensen_rank'] + portfolios['Trey_rank'] + portfolios['Sharpe_rank'] + portfolios['Vol_rank']
portfolios.sort_values(by=['Highest_rank'], ascending = True)

## Portfolio Analysis

In [ ]:
#Maximum Return Portofolio
max_ret_port = portfolios.iloc[portfolios['Returns'].idxmax()]
# idxmax() gives us the maximum value in the column specified.                               
max_ret_port

In [ ]:
#Minimum Volatility Portofolio
min_vol_port = portfolios.iloc[portfolios['Volatility'].idxmin()]
# idxmin() gives us the minimum value in the column specified.                               
min_vol_port

In [ ]:
#Highest Sharpe Ratio Portofolio
max_sharpe_port = portfolios.iloc[portfolios['Sharpe'].idxmax()]
max_sharpe_port

In [ ]:
#Highest Treynor Ratio Portofolio
max_tre_port = portfolios.iloc[portfolios['Treynor'].idxmax()]
# idxmin() gives us the minimum value in the column specified.                               
max_tre_port

In [ ]:
#Highest Jensen's alpha Portofolio
max_jen_port = portfolios.iloc[portfolios['Jensen'].idxmax()]
# idxmin() gives us the minimum value in the column specified.                               
max_jen_port

In [ ]:
#Highest overall rank
high_all_port = portfolios.iloc[portfolios['Highest_rank'].idxmin()]
# idxmin() gives us the minimum value in the column specified.                               
high_all_port

In [ ]:
#Overview and Comparison
Overview = pd.concat([min_vol_port, max_sharpe_port, max_tre_port, max_jen_port, high_all_port ], axis=1)
#Overview.iloc[-2].rename('new_name', inplace=True)
Overview.rename(columns={Overview.columns[0]: "Min Volatility"}, inplace=True)
Overview.rename(columns={Overview.columns[1]: "Max Sharpe"}, inplace=True)
Overview.rename(columns={Overview.columns[2]: "Max Treynor"}, inplace=True)
Overview.rename(columns={Overview.columns[3]: "Max Jensen Alpha"}, inplace=True)
Overview.rename(columns={Overview.columns[4]: "High in overall methods"}, inplace=True)
Overview

## Next Steps: Portfolio prediction testing

Now that the portfolios arebuilt, they are left to be tested. 

The test period is chosen starting from March 2015 to March 2018. 

The portfolios were built with optimization regarding one specific ratio or index, namely,
- Min Volatility
- Max Sharpe Ratio
- Max Treynor Ratio
- Max Jensn Alpha
- Max overall Score

By comparing the daily return using the weights in each portfolio, we are able to obverve Max Jensen portfolio has a much better return than other portfolios. Yet, such high return is brought with high risks, as what we have seen above in the corresponding volatilities of each portfolio. 
These portfolios are optimized based on the index or ratio they were built, so there is no easy answer which one is significantly better than the other. The investor has to choose based on their risk acceptances. 


### Back Testing

In [ ]:
#adjust of the original data
Oil_df_final=Oil_df_final[["TICKER", "PRC","date"]]
oil_table = Oil_df_final.pivot_table(index='date', columns='TICKER', values='PRC', aggfunc='sum').sort_values('date', ascending=True)
oil_table = oil_table.reset_index()

In [ ]:
def backtest_date(start,end):
    return start, end
backtest_date = backtest_date(20150301,20180301)

oil_table_bt = oil_table[(oil_table['date'] >= backtest_date[0]) & (oil_table['date'] <= backtest_date[1])]
oil_table_bt

In [ ]:
#Performance of Min Volatility Portofolio. 
oil_table_bt['Min_Volatility_PRC'] = (
    100*(oil_table_bt['CLR']/oil_table_bt['CLR'].shift(1)-1) * min_vol_port.iloc[2] +
    100*(oil_table_bt['COP']/oil_table_bt['COP'].shift(1)-1) * min_vol_port.iloc[3] +
    100*(oil_table_bt['CVX']/oil_table_bt['CVX'].shift(1)-1) * min_vol_port.iloc[4] +
    100*(oil_table_bt['EOG']/oil_table_bt['EOG'].shift(1)-1) * min_vol_port.iloc[5] +
    100*(oil_table_bt['FANG']/oil_table_bt['FANG'].shift(1)-1) * min_vol_port.iloc[6] +
    100*(oil_table_bt['MRO']/oil_table_bt['MRO'].shift(1)-1) * min_vol_port.iloc[7] +
    100*(oil_table_bt['OVV']/oil_table_bt['OVV'].shift(1)-1) * min_vol_port.iloc[8] +
    100*(oil_table_bt['OXY']/oil_table_bt['OXY'].shift(1)-1) * min_vol_port.iloc[9] +
    100*(oil_table_bt['PXD']/oil_table_bt['PXD'].shift(1)-1) * min_vol_port.iloc[10] +
    100*(oil_table_bt['RF']/oil_table_bt['RF'].shift(1)-1) * min_vol_port.iloc[11] +
    100*(oil_table_bt['XOM']/oil_table_bt['XOM'].shift(1)-1) * min_vol_port.iloc[12])
oil_table_bt

In [ ]:
#Performance of Max Sharpe Portofolio
oil_table_bt['Max_Sharpe_PRC'] = (
    100*(oil_table_bt['CLR']/oil_table_bt['CLR'].shift(1)-1) * max_sharpe_port.iloc[2] +
    100*(oil_table_bt['COP']/oil_table_bt['COP'].shift(1)-1) * max_sharpe_port.iloc[3] +
    100*(oil_table_bt['CVX']/oil_table_bt['CVX'].shift(1)-1) * max_sharpe_port.iloc[4] +
    100*(oil_table_bt['EOG']/oil_table_bt['EOG'].shift(1)-1) * max_sharpe_port.iloc[5] +
    100*(oil_table_bt['FANG']/oil_table_bt['FANG'].shift(1)-1) * max_sharpe_port.iloc[6] +
    100*(oil_table_bt['MRO']/oil_table_bt['MRO'].shift(1)-1) * max_sharpe_port.iloc[7] +
    100*(oil_table_bt['OVV']/oil_table_bt['OVV'].shift(1)-1) * max_sharpe_port.iloc[8] +
    100*(oil_table_bt['OXY']/oil_table_bt['OXY'].shift(1)-1) * max_sharpe_port.iloc[9] +
    100*(oil_table_bt['PXD']/oil_table_bt['PXD'].shift(1)-1) * max_sharpe_port.iloc[10] +
    100*(oil_table_bt['RF']/oil_table_bt['RF'].shift(1)-1) * max_sharpe_port.iloc[11] +
    100*(oil_table_bt['XOM']/oil_table_bt['XOM'].shift(1)-1) * max_sharpe_port.iloc[12])
oil_table_bt

In [ ]:
#Performance of Max Treynor	Portofolio
oil_table_bt['Max_Treynor_PRC'] = (
    100*(oil_table_bt['CLR']/oil_table_bt['CLR'].shift(1)-1) * max_tre_port.iloc[2] +
    100*(oil_table_bt['COP']/oil_table_bt['COP'].shift(1)-1) * max_tre_port.iloc[3] +
    100*(oil_table_bt['CVX']/oil_table_bt['CVX'].shift(1)-1) * max_tre_port.iloc[4] +
    100*(oil_table_bt['EOG']/oil_table_bt['EOG'].shift(1)-1) * max_tre_port.iloc[5] +
    100*(oil_table_bt['FANG']/oil_table_bt['FANG'].shift(1)-1) * max_tre_port.iloc[6] +
    100*(oil_table_bt['MRO']/oil_table_bt['MRO'].shift(1)-1) * max_tre_port.iloc[7] +
    100*(oil_table_bt['OVV']/oil_table_bt['OVV'].shift(1)-1) * max_tre_port.iloc[8] +
    100*(oil_table_bt['OXY']/oil_table_bt['OXY'].shift(1)-1) * max_tre_port.iloc[9] +
    100*(oil_table_bt['PXD']/oil_table_bt['PXD'].shift(1)-1) * max_tre_port.iloc[10] +
    100*(oil_table_bt['RF']/oil_table_bt['RF'].shift(1)-1) * max_tre_port.iloc[11] +
    100*(oil_table_bt['XOM']/oil_table_bt['XOM'].shift(1)-1) * max_tre_port.iloc[12])
oil_table_bt

In [ ]:
#Performance of Max Jensen Alpha Portofolio
oil_table_bt['Max_Jensen_PRC'] = (
    100*(oil_table_bt['CLR']/oil_table_bt['CLR'].shift(1)-1) * max_jen_port.iloc[2] +
    100*(oil_table_bt['COP']/oil_table_bt['COP'].shift(1)-1) * max_jen_port.iloc[3] +
    100*(oil_table_bt['CVX']/oil_table_bt['CVX'].shift(1)-1) * max_jen_port.iloc[4] +
    100*(oil_table_bt['EOG']/oil_table_bt['EOG'].shift(1)-1) * max_jen_port.iloc[5] +
    100*(oil_table_bt['FANG']/oil_table_bt['FANG'].shift(1)-1) * max_jen_port.iloc[6] +
    100*(oil_table_bt['MRO']/oil_table_bt['MRO'].shift(1)-1) * max_jen_port.iloc[7] +
    100*(oil_table_bt['OVV']/oil_table_bt['OVV'].shift(1)-1) * max_jen_port.iloc[8] +
    100*(oil_table_bt['OXY']/oil_table_bt['OXY'].shift(1)-1) * max_jen_port.iloc[9] +
    100*(oil_table_bt['PXD']/oil_table_bt['PXD'].shift(1)-1) * max_jen_port.iloc[10] +
    100*(oil_table_bt['RF']/oil_table_bt['RF'].shift(1)-1) * max_jen_port.iloc[11] +
    100*(oil_table_bt['XOM']/oil_table_bt['XOM'].shift(1)-1) * max_jen_port.iloc[12])
oil_table_bt

In [ ]:
#Performance of High in overall methods Portofolio
oil_table_bt['High_overall_PRC'] = (
    100*(oil_table_bt['CLR']/oil_table_bt['CLR'].shift(1)-1) * high_all_port.iloc[2] +
    100*(oil_table_bt['COP']/oil_table_bt['COP'].shift(1)-1) * high_all_port.iloc[3] +
    100*(oil_table_bt['CVX']/oil_table_bt['CVX'].shift(1)-1) * high_all_port.iloc[4] +
    100*(oil_table_bt['EOG']/oil_table_bt['EOG'].shift(1)-1) * high_all_port.iloc[5] +
    100*(oil_table_bt['FANG']/oil_table_bt['FANG'].shift(1)-1) * high_all_port.iloc[6] +
    100*(oil_table_bt['MRO']/oil_table_bt['MRO'].shift(1)-1) * high_all_port.iloc[7] +
    100*(oil_table_bt['OVV']/oil_table_bt['OVV'].shift(1)-1) * high_all_port.iloc[8] +
    100*(oil_table_bt['OXY']/oil_table_bt['OXY'].shift(1)-1) * high_all_port.iloc[9] +
    100*(oil_table_bt['PXD']/oil_table_bt['PXD'].shift(1)-1) * high_all_port.iloc[10] +
    100*(oil_table_bt['RF']/oil_table_bt['RF'].shift(1)-1) * high_all_port.iloc[11] +
    100*(oil_table_bt['XOM']/oil_table_bt['XOM'].shift(1)-1) * high_all_port.iloc[12])
oil_table_bt

In [ ]:
oil_table_bt

In [ ]:
#Plot the comparsion
oil_table_bt['date'] = pd.to_datetime(oil_table_bt['date'], format='%Y%m%d') #change the date types
oil_table_bt.index = oil_table_bt['date']
# Resample the data by year
oil_table_bt = oil_table_bt.resample('3M').mean()

oil_table_bt = oil_table_bt.reset_index()
# create the figure and axes objects
fig, ax = plt.subplots()
# plot each column as a separate line
oil_table_bt.plot(x='date', y='Min_Volatility_PRC', ax=ax, label='Min Volatility')
oil_table_bt.plot(x='date', y='Max_Sharpe_PRC', ax=ax, label='Max Sharpe')
oil_table_bt.plot(x='date', y='Max_Treynor_PRC', ax=ax, label='Max Treynor')
oil_table_bt.plot(x='date', y='Max_Jensen_PRC', ax=ax, label='Max Jensen')
oil_table_bt.plot(x='date', y='High_overall_PRC', ax=ax, label='High Overall')

# add a legend
ax.legend()

# show the plot
plt.show()